In [3]:
import flickrapi
api_key = '60624b0963634d316ccb8a12abdfc644'
api_secret = '3678327259a167f1'
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

In [4]:
photos = flickr.photos.search(tags='Travel',extras='tags', per_page=20)
total_photos = photos["photos"]["total"]

print(f"Number of travel photos: {total_photos}")

Number of travel photos: 92471


In [1]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
#from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image
from torch.utils.data import Dataset
import random
from tqdm import tqdm
import json
import threading


In [1]:
"""# Define the transformation to apply to the image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()]
    )
for photo in photos['photos']['photo']:
    photo_url = 'https://farm{}.staticflickr.com/{}/{}_{}.jpg'.format(photo['farm'], photo['server'], photo['id'], photo['secret'])
    if photo_url:
        response = requests.get(photo_url)
        img = Image.open(BytesIO(response.content))

        # Apply the transformation to the image
        img = transform(img)

        # Display the image
        plt.imshow(img.permute(1, 2, 0))
        plt.show()"""

"# Define the transformation to apply to the image\ntransform = transforms.Compose([\n    transforms.Resize(256),\n    transforms.CenterCrop(224),\n    transforms.ToTensor()]\n    )\nfor photo in photos['photos']['photo']:\n    photo_url = 'https://farm{}.staticflickr.com/{}/{}_{}.jpg'.format(photo['farm'], photo['server'], photo['id'], photo['secret'])\n    if photo_url:\n        response = requests.get(photo_url)\n        img = Image.open(BytesIO(response.content))\n\n        # Apply the transformation to the image\n        img = transform(img)\n\n        # Display the image\n        plt.imshow(img.permute(1, 2, 0))\n        plt.show()"

In [ ]:
train_ratio = 1
test_ratio = 0
search_tag='Vacation'
# Specify the number of photos to download
num_photos = 25000

# Initialize a list to store the downloaded photos
downloaded_photos = []

# Make multiple API requests to download the photos
for page in range(1, (num_photos // 500) + 2):
    # Construct the API request URL with the page parameter
    url = "https://www.flickr.com/services/rest/?method=flickr.photos.search&api_key={}&tags={}&per_page=500&page={}&format=json&nojsoncallback=1".format(api_key, search_tag, page)

    # Make the API request and parse the response
    response = requests.get(url,timeout=10)
    response_dict = json.loads(response.content.decode())

    # Extract the photo objects from the response and add them to the downloaded_photos list
    downloaded_photos.extend(response_dict['photos']['photo'])

    # Break the loop if the requested number of photos have been downloaded
    if len(downloaded_photos) >= num_photos:
        break

In [ ]:
output_dir='data/'
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")
#os.makedirs(train_dir, exist_ok=True)
#os.makedirs(test_dir, exist_ok=True)

# Create caption files for train and test sets
train_captions_file = os.path.join(output_dir, "train_captions.txt")
test_captions_file = os.path.join(output_dir, "test_captions.txt")
"""with open(train_captions_file, "w") as f:
    pass
with open(test_captions_file, "w") as f:
    pass
"""

In [ ]:
for photo in tqdm(downloaded_photos[10000:11000]):
    photo_url = 'https://farm{}.staticflickr.com/{}/{}_{}.jpg'.format(photo['farm'], photo['server'], photo['id'], photo['secret'])
    # Create the filename
    filename = "{}_{}.jpg".format(photo['id'], photo['secret'])
    img_path = os.path.join(output_dir, filename)
    
    train_img_path = os.path.join(train_dir, filename)
    test_img_path = os.path.join(test_dir, filename)

    if os.path.exists(train_img_path) or os.path.exists(test_img_path):
        print("Skipping {}, already downloaded".format(filename))
        continue
    
    # Download the image
    response = requests.get(photo_url)
    img1 = Image.open(BytesIO(response.content))
    
    # Apply the transformation to the image
    img = transform(img1)
    # Generate the image caption
    generated_text = image_to_text(photo_url)[0]['generated_text']
    
    # Choose a random number between 0 and 1 to determine which set the image belongs to
    r = random.random()
    if r < train_ratio:
        # Save the image to the train directory
        img_path = os.path.join(train_dir, filename)
        with open(train_captions_file, "a") as f:
            f.write("{}\t{}\n".format(filename, generated_text))
    else:
        # Save the image to the test directory
        img_path = os.path.join(test_dir, filename)
        with open(test_captions_file, "a") as f:
            f.write("{}\t{}\n".format(filename, generated_text))
    
    try:
        torchvision.utils.save_image(img, img_path)
    except FileNotFoundError as e:
        print("Error: File not found: {}".format(e))
        continue


In [ ]:
import zipfile

output_filename = "data.zip"

with zipfile.ZipFile(output_filename, "w") as zip_file:
    # Add the train directory and all its contents to the zip file
    for root, dirs, files in os.walk(train_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zip_file.write(file_path, os.path.relpath(file_path, train_dir))

    # Add the test directory and all its contents to the zip file
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zip_file.write(file_path, os.path.relpath(file_path, test_dir))

    # Add the train and test captions files to the zip file
    zip_file.write(train_captions_file, os.path.basename(train_captions_file))
    zip_file.write(test_captions_file, os.path.basename(test_captions_file))